In [1]:
from agents import Agent, Runner
from agents.mcp import MCPServerStdio
import os 
import json
import sys 

# Check if API keys are set
if not os.environ.get("ATLA_API_KEY"):
    raise ValueError("Please set the ATLA_API_KEY environment variable.")
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

In [2]:
async def run_agent_with_atla(user_request):
    async with MCPServerStdio(
            params={
                "command": "python",
                "args": ["../atla-mcp-server.py"],
                "env": {"ATLA_API_KEY":os.environ.get("ATLA_API_KEY")}
            }
        ) as atla_mcp_server:
            # Create an agent with the Atla evaluation server
            agent = Agent(
                name="AssistantWithAtlaEval",
                instructions="""
                You are a helpful assistant. Your goal is to provide high-quality responses to user requests.
                You can use the Atla evaluation tool to improve your responses.
                """,
                mcp_servers=[atla_mcp_server], # You can equip any Agent with Atla's MCP server like this
                model="gpt-4o-mini"
            )
            
            # Run the agent with the user request
            result = await Runner.run(agent, user_request)
            
            # Print the MCP call result
            for item in result.new_items:
                if item.type == "message_output_item":
                    print("\n*******  Agent's response: ******")
                    print(item.raw_item.content[0].text)
                elif item.type == 'tool_call_item':
                    print("\n*******  Agent's request to Atla tool: ******:")
                    print(json.loads(item.raw_item.arguments))
                elif item.type == 'tool_call_output_item':
                    print("\n*******  Atla's response to Agent's request: ******:")
                    print(json.loads(item.output)['text'])

In [3]:
await run_agent_with_atla(user_request="Write a one-line poem about the ocean. Evaluate it for cliche and improve it once using the feedback.")


*******  Agent's request to Atla tool: ******:
{'model_input': 'Write a one-line poem about the ocean.', 'model_output': 'The ocean whispers secrets to the shore.', 'evaluation_criteria': 'Assess the poem for originality and whether it falls into cliché territory, suggesting improvements if necessary.', 'expected_model_output': 'The poem is somewhat cliché as the concept of the ocean "whispering secrets" is a common metaphor in poetry. Suggest using more unique imagery or an unexpected perspective on the ocean.'}

*******  Atla's response to Agent's request: ******:
{"score": "4", "critique": "The poem is somewhat clich\u00e9 as the concept of the ocean \"whispering secrets\" is a common metaphor in poetry. Suggest using more unique imagery or an unexpected perspective on the ocean. To improve, consider adding sensory details or exploring the emotional resonance of the ocean's presence. For instance, the poem could describe the sound of the waves, the smell of the saltwater, or the fe